# Sondera Harness + LangGraph

**Add policy enforcement to your LangGraph agents.**

This notebook shows how to use Sondera Harness with LangGraph. You'll define LangChain tools, write Cedar policies, and see how the harness evaluates each action before it executes.

We'll build a financial assistant that:
- **Allows** viewing portfolio and account information
- **Allows** processing refunds up to \$10,000
- **Blocks** refunds over \$10,000 (requires manager approval)
- **Blocks** requests for sensitive PII like SSN or credit card numbers

*This is a simulation: we evaluate what would be allowed, but no actual LLM calls are made.*

## Install

In [ ]:
%%capture
%pip install "sondera-harness[langgraph]"

## Define LangChain Tools

First, define your tools using LangChain's `@tool` decorator. Sondera automatically analyzes these tools to build the Cedar schema.

In [ ]:
from langchain_core.tools import tool


@tool
def get_portfolio(customer_id: str) -> str:
    """Return the customer's portfolio holdings and current performance."""
    return f"Portfolio for {customer_id}: AAPL (100 shares), GOOGL (50 shares)"


@tool
def get_account_info(customer_id: str) -> str:
    """Return customer account details including risk tolerance and account type."""
    return f"Account {customer_id}: Type=Brokerage, Risk=Moderate"


@tool
def process_refund(customer_id: str, amount: int, reason: str) -> str:
    """Process a refund for a customer. Amount in dollars, must be positive."""
    return f"Refund of ${amount} processed for {customer_id}: {reason}"


@tool
def send_notification(customer_id: str, subject: str, message: str) -> str:
    """Send a notification or email to the customer."""
    return f"Notification sent to {customer_id}: {subject}"


tools = [get_portfolio, get_account_info, process_refund, send_notification]
print(f"Defined {len(tools)} tools: {[t.name for t in tools]}")

## Create Agent Metadata

Use `create_agent_from_langchain_tools` to extract tool metadata for policy evaluation. This analyzes your LangChain tools and creates the schema that Cedar needs to evaluate policies.

In [ ]:
from sondera.langgraph import create_agent_from_langchain_tools

agent_metadata = create_agent_from_langchain_tools(
    tools=tools,
    agent_id="financial-assistant",
    agent_name="Financial Assistant",
    agent_description="A financial assistant that helps customers with portfolio management, account inquiries, and refund processing.",
)

print(
    f"Agent metadata for '{agent_metadata.name}' with {len(agent_metadata.tools)} tools"
)
for t in agent_metadata.tools:
    print(f"  - {t.name}: {len(t.parameters)} parameters")

## Define the Policy

Write Cedar policies to control what the agent can do. The namespace `Financial_Assistant` is derived from the agent name (spaces become underscores).

**Pattern:** Permit by default, then forbid specific things. `forbid` always wins over `permit`.

In [ ]:
from cedar import PolicySet

policy_set = PolicySet("""
// Allow viewing portfolio information
@id("allow-get-portfolio")
permit(principal, action == Financial_Assistant::Action::"get_portfolio", resource);

// Allow viewing account information
@id("allow-get-account-info")
permit(principal, action == Financial_Assistant::Action::"get_account_info", resource);

// Allow sending notifications
@id("allow-send-notification")
permit(principal, action == Financial_Assistant::Action::"send_notification", resource);

// Allow refunds by default (restricted below)
@id("allow-process-refund")
permit(principal, action == Financial_Assistant::Action::"process_refund", resource);

// Block refunds over $10,000 - requires manager approval
@id("forbid-large-refund")
forbid(principal, action == Financial_Assistant::Action::"process_refund", resource)
when {
  context has parameters &&
  context.parameters.amount > 10000
};

// Block notifications that might contain PII
@id("forbid-pii-in-notification")
forbid(principal, action == Financial_Assistant::Action::"send_notification", resource)
when {
  context has parameters &&
  (context.parameters.message like "*SSN*" ||
   context.parameters.message like "*social security*" ||
   context.parameters.message like "*credit card*")
};
""")

print(f"Policy loaded with {len(policy_set)} rules")

## Initialize the Harness

Create a `CedarPolicyHarness` for local policy evaluation. In production, you'd use `SonderaRemoteHarness` to connect to the Sondera Platform for centralized policy management.

In [ ]:
from sondera import CedarPolicyHarness
from sondera.harness.cedar.schema import agent_to_cedar_schema

schema = agent_to_cedar_schema(agent_metadata)
harness = CedarPolicyHarness(policy_set=policy_set, schema=schema)
await harness.initialize(agent=agent_metadata)

print("Harness initialized and ready")

## How the Middleware Works

In a real LangGraph agent, `SonderaHarnessMiddleware` intercepts execution at each stage:

| LangGraph Hook | Stage | What It Checks |
|:---------------|:------|:---------------|
| `abefore_agent` | `PRE_RUN` | Session start, user input |
| `awrap_model_call` | `PRE_MODEL` / `POST_MODEL` | Model requests and responses |
| `awrap_tool_call` | `PRE_TOOL` / `POST_TOOL` | Tool arguments and results |
| `aafter_agent` | `POST_RUN` | Session end, finalize trajectory |

Below, we'll simulate what the middleware does by calling `adjudicate()` directly.

## Run It

Let's test what the agent is allowed to do. We call `adjudicate()` for each action, simulating what the middleware would do at the `PRE_TOOL` stage.

In [ ]:
from sondera import Decision, Role, Stage, ToolRequestContent

# ANSI color codes
GREEN = "\033[92m"
RED = "\033[91m"
RESET = "\033[0m"


def check(description: str, result, expected: Decision):
    """Check result matches expected and print colored status."""
    color = GREEN if result.decision == Decision.ALLOW else RED
    status = result.decision.name
    icon = "\u2705" if result.decision == expected else "\u274c"
    print(f"{icon} {description} ({color}{status}{RESET})")
    if result.decision != expected:
        print(f"   Expected {expected.name}, got {result.decision.name}")
        if result.reason:
            print(f"   Reason: {result.reason}")


# =============================================================================
# ALLOWED OPERATIONS
# =============================================================================

# Viewing portfolio is allowed
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(tool_id="get_portfolio", args={"customer_id": "CUST001"}),
)
check("View customer portfolio", result, Decision.ALLOW)

# Viewing account info is allowed
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(tool_id="get_account_info", args={"customer_id": "CUST001"}),
)
check("View account info", result, Decision.ALLOW)

# Small refund is allowed
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="process_refund",
        args={"customer_id": "CUST001", "amount": 500, "reason": "Duplicate charge"},
    ),
)
check("Process $500 refund", result, Decision.ALLOW)

# Refund at the limit is allowed
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="process_refund",
        args={"customer_id": "CUST001", "amount": 10000, "reason": "Service issue"},
    ),
)
check("Process $10,000 refund (at limit)", result, Decision.ALLOW)

# Normal notification is allowed
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="send_notification",
        args={
            "customer_id": "CUST001",
            "subject": "Refund Processed",
            "message": "Your refund of $500 has been processed.",
        },
    ),
)
check("Send normal notification", result, Decision.ALLOW)

# =============================================================================
# BLOCKED OPERATIONS
# =============================================================================

# Large refund is blocked
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="process_refund",
        args={
            "customer_id": "CUST001",
            "amount": 15000,
            "reason": "Customer complaint",
        },
    ),
)
check("Process $15,000 refund (over limit)", result, Decision.DENY)

# Notification with SSN is blocked
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="send_notification",
        args={
            "customer_id": "CUST001",
            "subject": "Account Verification",
            "message": "Please confirm your SSN: 123-45-6789",
        },
    ),
)
check("Send notification with SSN", result, Decision.DENY)

# Notification mentioning credit card is blocked
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="send_notification",
        args={
            "customer_id": "CUST001",
            "subject": "Payment Info",
            "message": "Your credit card ending in 4242 was charged.",
        },
    ),
)
check("Send notification with credit card info", result, Decision.DENY)

print(f"\n{GREEN}{'=' * 50}")
print(f"All policy evaluations complete!{RESET}")

## Using the Middleware

In a real LangGraph agent, you'd use `SonderaHarnessMiddleware` to automatically enforce policies. Here's how:

```python
from langchain.agents import create_agent
from sondera.langgraph import SonderaHarnessMiddleware, Strategy

# Create the middleware
middleware = SonderaHarnessMiddleware(
    harness=harness,
    strategy=Strategy.BLOCK,  # or Strategy.STEER
)

# Create the agent with middleware
agent = create_agent(
    model=my_llm,  # Your LLM (OpenAI, Anthropic, etc.)
    tools=tools,
    middleware=[middleware],
)

# Run the agent - policies are automatically enforced
result = await agent.ainvoke({"messages": [...]})
```

**Strategies:**
- `Strategy.BLOCK`: Stop execution immediately on policy violation
- `Strategy.STEER`: Continue execution with modified content, letting the model adapt

## Try It Yourself

Experiment with the policy. Go back to **Define the Policy** and try:

- **Change the refund limit**: Try `context.parameters.amount > 5000` instead of `10000`
- **Add more PII patterns**: Add `context.parameters.message like "*account number*"`
- **Block specific customers**: Add a rule that forbids actions for `customer_id == "BLOCKED_USER"`

Or modify the test cases below:

In [ ]:
def show(description: str, result):
    """Print colored result."""
    color = GREEN if result.decision == Decision.ALLOW else RED
    print(f"{description}: {color}{result.decision.name}{RESET}")


# Try a refund right at the boundary
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="process_refund",
        args={
            "customer_id": "CUST001",
            "amount": 10001,  # Just over the limit
            "reason": "Edge case test",
        },
    ),
)
show("Process $10,001 refund", result)

# Try different PII patterns
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="send_notification",
        args={
            "customer_id": "CUST001",
            "subject": "Verification",
            "message": "Please confirm your social security number for verification.",
        },
    ),
)
show("Notification asking for social security", result)

# This should be allowed (no PII)
result = await harness.adjudicate(
    Stage.PRE_TOOL,
    Role.MODEL,
    ToolRequestContent(
        tool_id="send_notification",
        args={
            "customer_id": "CUST001",
            "subject": "Welcome",
            "message": "Welcome to our platform! Your account is now active.",
        },
    ),
)
show("Welcome notification", result)

## Next Steps

- [Quickstart](https://docs.sondera.ai/quickstart/) - Core concepts without LangGraph
- [Writing Policies](https://docs.sondera.ai/writing-policies/) - Cedar syntax and patterns
- [ADK Integration](https://docs.sondera.ai/integrations/adk/) - Use with Google's Agent Development Kit
- [Strands Integration](https://docs.sondera.ai/integrations/strands/) - Use with AWS Strands